In [1]:
from IPython import display
import ultralytics
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
display.clear_output()
ultralytics.checks()

Ultralytics YOLOv8.2.99  Python-3.11.7 torch-2.2.0+cpu CPU (AMD Ryzen 7 5825U with Radeon Graphics)
Setup complete  (16 CPUs, 15.3 GB RAM, 48.0/238.0 GB disk)


In [13]:
model = YOLO("yolov8n-seg.yaml").load("yolov8n-seg.pt")

Transferred 417/417 items from pretrained weights


In [3]:
model = YOLO('./runs/segment/train7/weights/best.pt')

In [ ]:
# freeze 10 layers
freeze = 0

freeze = [f'model.{x}.' for x in range(freeze)] # สร้างลิสต์ของชื่อเลเยอร์ที่ต้องการล็อค
for k, v in model.named_parameters(): # วนลูปผ่านทุกพารามิเตอร์ในโมเดล
    
    # เช็คว่าชื่อของพารามิเตอร์ (k) มีคำว่า model.{x}. ใด ๆ ในลิสต์ freeze หรือไม่
    if any(x in k for x in freeze):
        print(f'freezing {k}')
        v.requires_grad = False # ปิดการอัปเดตน้ำหนักของเลเยอร์นั้น
    else:
        print(f'unfreeze {k}')
        v.requires_grad = True

In [7]:
import torch

optimizer = torch.optim.SGD(
    filter(lambda p: p.requires_grad, model.parameters()), # เลือกพารามิเตอร์ของโมเดลที่ไม่ได้ล็อค เพื่อ update ค่า weight
    lr=1e-4,
    momentum=0.9 
)

In [8]:
# train model
result = model.train(
    batch=-1,
    device="cpu",
    data="./setting.yaml",
    epochs=100,
    imgsz=640,
    cache=True
)

New https://pypi.org/project/ultralytics/8.3.25 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.99  Python-3.11.7 torch-2.2.0+cpu CPU (AMD Ryzen 7 5825U with Radeon Graphics)
engine\trainer: task=segment, mode=train, model=./runs/segment/train7/weights/best.pt, data=./setting.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=True, device=cpu, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, emb

train: Scanning D:\Train project\General-Test\dataset\train\labels.cache... 420 images, 0 backgrounds, 0 corrupt: 100%|██████████| 420/420 [00:00<?, ?it/s]


WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.4GB RAM): 100%|██████████| 420/420 [00:01<00:00, 354.52it/s]
val: Scanning D:\Train project\General-Test\dataset\val\labels.cache... 140 images, 0 backgrounds, 0 corrupt: 100%|██████████| 140/140 [00:00<?, ?it/s]


WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

    100/100         0G     0.2053     0.3594     0.2118     0.8337          8        640: 100%|██████████| 27/27 [03:09<00:00,  7.03s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:23<00:00,  4.67s/it]

                   all        140        208      0.937      0.897      0.942      0.887      0.934      0.894      0.941      0.876



100 epochs completed in 6.031 hours.
Optimizer stripped from runs\segment\train8\weights\last.pt, 6.8MB
Optimizer stripped from runs\segment\train8\weights\best.pt, 6.8MB

Validating runs\segment\train8\weights\best.pt...
Ultralytics YOLOv8.2.99  Python-3.11.7 torch-2.2.0+cpu CPU (AMD Ryzen 7 5825U with Radeon Graphics)
YOLOv8n-seg summary (fused): 195 layers, 3,259,429 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:20<00:00,  4.02s/it]


                   all        140        208      0.922      0.919       0.96      0.896      0.922      0.919       0.96      0.882
                   box         20         20      0.964          1      0.995      0.991      0.964          1      0.995      0.988
                  coin         20         23      0.971          1      0.995      0.966      0.971          1      0.995       0.98
                 glass         20         27      0.942      0.963      0.984       0.89      0.942      0.963      0.984      0.896
                  mask         20         20      0.915       0.55       0.81      0.608      0.915       0.55       0.81      0.609
               sandals         20         50      0.876      0.992       0.99      0.966      0.876      0.992       0.99       0.96
                  sock         20         48      0.788      0.938       0.95      0.922      0.788      0.938      0.953      0.892
                 spoon         20         20          1      0.993   

In [ ]:
# train model ต่อจากที่หยุด
model = YOLO('./runs/segment/train7/weights/last.pt')
results = model.train(resume = True)

In [ ]:
# load model
model = YOLO('./runs/segment/train8/weights/best.pt')

# แปลงโมเดลเป็น ncnn
model.export(format="ncnn")

In [ ]:
# Use Raspberry Pi Camera

rpicam-hello # test camera

# Inference with Camera
import cv2
from picamera2 import Picamera2

from ultralytics import YOLO

# Initialize the Picamera2
picam2 = Picamera2()
picam2.preview_configuration.main.size = (1280, 720)
picam2.preview_configuration.main.format = "RGB888"
picam2.preview_configuration.align()
picam2.configure("preview")
picam2.start()

# Load the YOLO11 model
model = YOLO("yolo11n.pt")

while True:
    # Capture frame-by-frame
    frame = picam2.capture_array()

    # Run YOLO11 inference on the frame
    results = model(frame)

    # Visualize the results on the frame
    annotated_frame = results[0].plot()

    # Display the resulting frame
    cv2.imshow("Camera", annotated_frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) == ord("q"):
        break

# Release resources and close windows
cv2.destroyAllWindows()